# Some attacks from misuse of block ciphers

It's important that you use the right mode of operation for a block cipher. It's also important that when you use a mode of operation, it is set-up correctly.

## ECB - Electronic Code Book

We're going to look at the Electronic Code Book (ECB) for AES to kick us off. That mode of operation is never secure regardless of which block-cipher you pick.

We start by just demoing the general capability.

First we need to set-up a key.

In [ ]:
from Crypto.Random import get_random_bytes

aes_key = get_random_bytes(16)


In [24]:
from Crypto.Cipher import AES

cipher = AES.new(aes_key, AES.MODE_ECB)

plaintext = b'Super Secret Msg'

cipherText = cipher.encrypt(plaintext)

print("Encrypted text:", cipherText)

plainText = cipher.decrypt(cipherText)

print("Recovered Plain Text:", plainText)



Encrypted text: b'\xf9\xf8\xeb\xfa%\xa7\t\x85n\x95x$\xa2sd\x87'
Recovered Plain Text: b'Super Secret Msg'


So what's the problem here?

The problem is the same plain-text encrypts to the same cipher text!

Let's show this is insecure using the game!

In [25]:
class AES_ECBGameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = b'Super Secret Msg'
                m1 = b'Super Secret Msg'
            else:
                m0 = b'Super Secret Msg'
                m1 = b'Super Urgent Msg'
            return m0, m1 

    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

Next we need the rules of the game. We're going to fix one key across all iterations of the game!

In [77]:
class AESFixedKey:
    from Crypto.Random import get_random_bytes

    def __init__(self, mode, iv=get_random_bytes(16)):
       self.key = get_random_bytes(16)
       self.aes = AES.new(self.key, mode, iv=iv)

    def encrypt(self, data):
        return self.aes.encrypt(data)

    def decrypt(self, data):
        return self.aes.decrypt(data)


And let's test it works how we want it to!

In [45]:
aesFixedKey = AESFixedKey(AES.MODE_ECB)
aesFixedKey.encrypt(plaintext)

b'1Qx\xcf\r\x9d\xfe\x0e\x8a\xfeSG?B\xf2\x12'

Right, let's play the game!

In [46]:
from SemanticSecurityGame import SemanticSecurityGame

strategy = AES_ECBGameStrategy()
game = SemanticSecurityGame(aesFixedKey, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}

# Cipher Block Chaining Mode - CBC.

There is a security proof showing that if CBC is used *correctly* the strength of the scheme reduces to the underlying strength of the cipher.

However, if you don't follow the conditions of the proof (i.e. that the IV is completely random) then terrible things happen. 

Let's explore!

In [92]:
iv=b'0000000000000000'
key=b'0000000000000000'

aes = AES.new(key,AES.MODE_CBC, iv=iv)
aes.encrypt(plaintext)

b'xJu\xa1\xf3\xb5\x85\xd9~\xbc/I\xd6c~\xc2'

So we learned with a fixed key and a fixed IV, it's the same value each time. How do we ensure that the IV is the same across multiple runs?

Suppose the IV is simply a counter. It counts up for each encryption!

In [91]:
iv=b'0000000000000001'
key=b'0000000000000000'

aes = AES.new(key,AES.MODE_CBC, iv=iv)
aes.encrypt(b'Super Secret Msf')

b'xJu\xa1\xf3\xb5\x85\xd9~\xbc/I\xd6c~\xc2'

Ooops!